In [1]:
using Distributed
using MachineLearningPotential
using BenchmarkTools
using StaticArrays

In [2]:
atoms = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]*18.8973*0.36258

positions = [SVector{3}(p[i] for i in 1:3) for p in atoms]
dis2mat = get_distance2_mat(positions)
X = [ 1    1              0.001   0.000  11.338
 1    0              0.001   0.000  11.338
 1    1              0.020   0.000  11.338
 1    0              0.020   0.000  11.338
 1    1              0.035   0.000  11.338
 1    0              0.035   0.000  11.338
 1    1              0.100   0.000  11.338
 1    0              0.100   0.000  11.338
 1    1              0.400   0.000  11.338
 1    0              0.400   0.000  11.338]

radsymmvec = []

for row in eachrow(X)
    symmfunc = RadialType2{Float64}(row[3],row[5],[row[1],row[2]])
    push!(radsymmvec,symmfunc)
end

V = [[0.0001,1,1,11.338],[0.0001,-1,2,11.338],[0.003,-1,1,11.338],[0.003,-1,2,11.338],[0.008,-1,1,11.338],[0.008,-1,2,11.228],[0.008,1,2,11.338],[0.015,1,1,11.338],[0.015,-1,2,11.338],[0.015,-1,4,11.338],[0.015,-1,16,11.338],[0.025,-1,1,11.338],[0.025,1,1,11.338],[0.025,1,2,11.338],[0.025,-1,4,11.338],[0.025,-1,16,11.338],[0.025,1,16,11.338],[0.045,1,1,11.338],[0.045,-1,2,11.338],[0.045,-1,4,11.338],[0.045,1,4,11.338],[0.045,1,16,11.338],[0.08,1,1,11.338],[0.08,-1,2,11.338],[0.08,-1,4,11.338],[0.08,1,4,11.338]]

T = [[1.,1.,1.],[1.,1.,0.],[1.,0.,0.]]

angularsymmvec = []

for element in V 
    for types in T
        symmfunc = AngularType3{Float64}(element[1],element[2],element[3],11.338,types)
        push!(angularsymmvec,symmfunc)
    end
end

total_symm_vec = vcat(radsymmvec,angularsymmvec)
f_mat = cutoff_function.(sqrt.(dis2mat),Ref(total_symm_vec[1].r_cut))

MethodError: MethodError: no method matching AngularType3{Float64}(::Float64, ::Float64, ::Float64, ::Float64, ::Vector{Float64})
Closest candidates are:
  AngularType3{T}(::Any, ::Any, ::Any, ::Any, ::Any, !Matched::Any) where T at ~/.julia/dev/MachineLearningPotential/src/SymmFunc.jl:55

In [3]:
calc_one_symm_val(r2_ij,fc_ij,eta) =  fc_ij*exp(-eta*r2_ij)

function calc_symm_function(positions,dist2_matrix,fc_matrix,index,symmfunc::RadialType2) 
    if symmfunc.type_vec == [1.,1.]   

        eta=symmfunc.eta
        #g_vec = calc_one_symm_val.(dist2_matrix[:,index],fc_matrix[:,index],eta)
        g_vec = [calc_one_symm_val.(dist2_matrix[jindex,index],fc_matrix[jindex,index],eta) for jindex in eachindex(positions) if jindex != index]
        
        return sum(g_vec)
    else
        return 0.
    end

end

function calc_one_symm_val(position1,position2,position3,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk,η,λ,ζ)
    θ = angular_measure(position1,position2,position3,r2_ij,r2_ik)
    g= (1+λ*θ)^ζ * exp(-η*(r2_ij+r2_ik+r2_jk)) * f_ij * f_ik * f_jk   
return g
end

function calc_symm_val(θ,r_sum,f_ij,f_ik,f_jk,symmfunc::AngularType3)
    if symmfunc.type_vec == [1.,1.,1.]
        η,λ,ζ = symmfunc.eta,symmfunc.lambda,symmfunc.zeta
        
        g = calc_symm_val(θ,r_sum,f_ij,f_ik,f_jk,η,λ,ζ)  
    else
        g = 0.
    end 
    return g
end

function calc_symm_val(θ,r_sum,f_ij,f_ik,f_jk,η,λ,ζ)
    g= (1+λ*θ)^ζ * exp(-η*(r_sum)) * f_ij * f_ik * f_jk    
    return g
end


calc_symm_val (generic function with 2 methods)

In [4]:
function calc_symm_functions(positions,dist2_mat,index,fc_mat,symmfuncs)
    
    gmat=Array{Float64}(undef,length(symmfuncs),size(dist2_mat)[1])

    #for index in eachindex(positions)
        gmat[:,index] = calc_symm_function.(Ref(positions),Ref(dist2_mat),Ref(fc_mat),Ref(index),symmfuncs)

    #end
return gmat
end

calc_symm_functions (generic function with 1 method)

In [5]:
@benchmark calc_symm_functions($positions,$dis2mat,$f_mat,$total_symm_vec)

UndefVarError: UndefVarError: f_mat not defined

In [6]:
calcfactor(zeta::Int64,value::Float64) = 2.0^(1-zeta)*value

@benchmark calcfactor($rand(-2:2),$rand())

BenchmarkTools.Trial: 10000 samples with 999 evaluations.
 Range (min … max):  11.011 ns … 46.515 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     11.856 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   11.906 ns ±  0.646 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

         ▁▄▅█▇██▆▆▇▇▇█▆▆█▆▆▇▅▅▆▅▆▅▄▃▂                          
  ▁▁▁▂▃▄▆████████████████████████████▇▇▆▅▅▅▃▃▃▂▂▂▂▂▁▁▂▂▂▁▂▁▁▂ ▅
  11 ns           Histogram: frequency by time        13.3 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [7]:
@benchmark angular_measure($positions[1],$positions[3],$positions[10],$dis2mat[1,3],$dis2mat[1,10])

BenchmarkTools.Trial: 10000 samples with 999 evaluations.
 Range (min … max):  11.078 ns … 36.498 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     11.174 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   11.394 ns ±  0.560 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▆█▅    ▂▆▆     ▂▅▅      ▂▃                                 ▂
  ▇███▅▅▆▅███▅▅▆▅▅███▅▃▅▃▄▄███▇▅▄▆▅▆▇▆▆▄▄▅▅▆▆▆▆▆▆▆▆▆▆▆▆▆▆▅▆▅▅ █
  11.1 ns      Histogram: log(frequency) by time      12.9 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.